![Piggy bank](piggy_bank.jpg)

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to `boolean` data type:<br> `1` if `"yes"`, otherwise `0` |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0` |

<br>

## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type:<br> `1` if `"success"`, otherwise `0`. |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0`. |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

In [220]:
import pandas as pd
import numpy as np

In [221]:
df = pd.read_csv("bank_marketing.csv")

# Chapter 1: Limpieza de datos

In [222]:
# Muestra del df
df.head()

,client_id,age,job,marital,education,credit_default,mortgage,month,day,contact_duration,number_contacts,previous_campaign_contacts,previous_outcome,cons_price_idx,euribor_three_months,campaign_outcome
0,0,56,housemaid,married,basic.4y,no,no,may,13,261,1,0,nonexistent,93.994,4.857,no
1,1,57,services,married,high.school,unknown,no,may,19,149,1,0,nonexistent,93.994,4.857,no
2,2,37,services,married,high.school,no,yes,may,23,226,1,0,nonexistent,93.994,4.857,no
3,3,40,admin.,married,basic.6y,no,no,may,27,151,1,0,nonexistent,93.994,4.857,no
4,4,56,services,married,high.school,no,no,may,3,307,1,0,nonexistent,93.994,4.857,no


In [223]:
# Información general del df, sus columnas y sus tipos de dato
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   client_id                   41188 non-null  int64  
 1   age                         41188 non-null  int64  
 2   job                         41188 non-null  object 
 3   marital                     41188 non-null  object 
 4   education                   41188 non-null  object 
 5   credit_default              41188 non-null  object 
 6   mortgage                    41188 non-null  object 
 7   month                       41188 non-null  object 
 8   day                         41188 non-null  int64  
 9   contact_duration            41188 non-null  int64  
 10  number_contacts             41188 non-null  int64  
 11  previous_campaign_contacts  41188 non-null  int64  
 12  previous_outcome            41188 non-null  object 
 13  cons_price_idx              411

In [224]:
# Verificamos que no tenga datos duplicados en la columna 'client_id'
duplicates = df.duplicated(subset=['client_id'])
if not df[duplicates].empty:
    print('There are duplicated values')
    print(df[duplicates])
else:
    print("There aren't duplicated values")

There aren't duplicated values


In [225]:
# Voy a crear los 3 DF que me solicitan a partir del DF original
client = df[['client_id', 'age', 'job', 'marital', 'education', 'credit_default', 'mortgage']].copy(deep=True)

campaign = df[['client_id', 'number_contacts', 'contact_duration', 'previous_campaign_contacts', 'previous_outcome', 'campaign_outcome', 'month', 'day']].copy(deep=True)

economics = df[['client_id', 'cons_price_idx', 'euribor_three_months']].copy(deep=True)

## Limpieza DF client:

In [226]:
# Revisamos cada una de las columnas para ver la distribución de los datos y si hay valores anómalos
for column in client.columns:
    print(f"{column}:")
    print(client[column].describe())
    print()

client_id:
count    41188.00000
mean     20593.50000
std      11890.09578
min          0.00000
25%      10296.75000
50%      20593.50000
75%      30890.25000
max      41187.00000
Name: client_id, dtype: float64

age:
count    41188.00000
mean        40.02406
std         10.42125
min         17.00000
25%         32.00000
50%         38.00000
75%         47.00000
max         98.00000
Name: age, dtype: float64

job:
count      41188
unique        12
top       admin.
freq       10422
Name: job, dtype: object

marital:
count       41188
unique          4
top       married
freq        24928
Name: marital, dtype: object

education:
count                 41188
unique                    8
top       university.degree
freq                  12168
Name: education, dtype: object

credit_default:
count     41188
unique        3
top          no
freq      32588
Name: credit_default, dtype: object

mortgage:
count     41188
unique        3
top         yes
freq      21576
Name: mortgage, dtype: object



In [227]:
# Revisamos cada una de las categorias de las columnas categoricas
for col in ['job', 'marital', 'education', 'credit_default', 'mortgage']:
    print(f"{col}:")
    print(client[col].value_counts())
    print()

job:
admin.           10422
blue-collar       9254
technician        6743
services          3969
management        2924
retired           1720
entrepreneur      1456
self-employed     1421
housemaid         1060
unemployed        1014
student            875
unknown            330
Name: job, dtype: int64

marital:
married     24928
single      11568
divorced     4612
unknown        80
Name: marital, dtype: int64

education:
university.degree      12168
high.school             9515
basic.9y                6045
professional.course     5243
basic.4y                4176
basic.6y                2292
unknown                 1731
illiterate                18
Name: education, dtype: int64

credit_default:
no         32588
unknown     8597
yes            3
Name: credit_default, dtype: int64

mortgage:
yes        21576
no         18622
unknown      990
Name: mortgage, dtype: int64



In [228]:
# Remplazando . por _ en job y education
client['job'] = client['job'].str.replace('.', '_')
client['education'] = client['education'].str.replace('.', '_')

In [229]:
# Remplazo valores unknown por valores NaN
client.loc[client['education'] == 'unknown', 'education'] = np.NaN

In [230]:
# Remplazando valores en credit_default
client["credit_default"] = np.where(df["credit_default"] == "yes", True, False)

In [231]:
# Remplazando valores en mortgage
client["mortgage"] = np.where(df["mortgage"] == "yes", True, False)

In [232]:
# Comparamos nuevamente la estructura de una columna
print(client['credit_default'].describe())
print()
print(client['credit_default'].value_counts())

count     41188
unique        2
top       False
freq      41185
Name: credit_default, dtype: object

False    41185
True         3
Name: credit_default, dtype: int64


In [233]:
# Comparamos nuevamente la estructura de una columna
print(client['mortgage'].describe())
print()
print(client['mortgage'].value_counts())

count     41188
unique        2
top        True
freq      21576
Name: mortgage, dtype: object

True     21576
False    19612
Name: mortgage, dtype: int64


In [234]:
print(client.dtypes)

client_id          int64
age                int64
job               object
marital           object
education         object
credit_default      bool
mortgage            bool
dtype: object


## Limpieza DF campaign:

In [235]:
# Revisamos cada una de las columnas para ver la distribución de los datos y si hay valores anómalos
for column in campaign.columns:
    print(f"{column}:")
    print(campaign[column].describe())
    print()

client_id:
count    41188.00000
mean     20593.50000
std      11890.09578
min          0.00000
25%      10296.75000
50%      20593.50000
75%      30890.25000
max      41187.00000
Name: client_id, dtype: float64

number_contacts:
count    41188.000000
mean         2.567593
std          2.770014
min          1.000000
25%          1.000000
50%          2.000000
75%          3.000000
max         56.000000
Name: number_contacts, dtype: float64

contact_duration:
count    41188.000000
mean       258.285010
std        259.279249
min          0.000000
25%        102.000000
50%        180.000000
75%        319.000000
max       4918.000000
Name: contact_duration, dtype: float64

previous_campaign_contacts:
count    41188.000000
mean         0.172963
std          0.494901
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          7.000000
Name: previous_campaign_contacts, dtype: float64

previous_outcome:
count           41188
unique              3
top   

In [236]:
# Revisamos cada una de las categorias de las columnas categoricas
for col in ['previous_outcome', 'campaign_outcome', 'month']:
    print(f"{col}:")
    print(campaign[col].value_counts())
    print()

previous_outcome:
nonexistent    35563
failure         4252
success         1373
Name: previous_outcome, dtype: int64

campaign_outcome:
no     36548
yes     4640
Name: campaign_outcome, dtype: int64

month:
may    13769
jul     7174
aug     6178
jun     5318
nov     4101
apr     2632
oct      718
sep      570
mar      546
dec      182
Name: month, dtype: int64



In [237]:
# Remplazando valores en mortgage
campaign["previous_outcome"] = np.where(df["previous_outcome"] == "success", True, False)

In [238]:
# Remplazando valores en mortgage
campaign["campaign_outcome"] = np.where(df["campaign_outcome"] == "yes", True, False)

In [239]:
campaign['day'] = campaign['day'].astype('str')

In [240]:
campaign['last_contact_date'] = '2022-' + campaign['month'] + '-' + campaign['day']

In [241]:
campaign['last_contact_date'] = pd.to_datetime(campaign['last_contact_date'], infer_datetime_format=True, errors='coerce')
campaign['last_contact_date'] = campaign['last_contact_date'].dt.strftime('%Y-%m-%d')

In [242]:
campaign = campaign.drop(['month', 'day'], axis=1)

In [243]:
campaign.head()

,client_id,number_contacts,contact_duration,previous_campaign_contacts,previous_outcome,campaign_outcome,last_contact_date
0,0,1,261,0,False,False,2022-05-13
1,1,1,149,0,False,False,2022-05-19
2,2,1,226,0,False,False,2022-05-23
3,3,1,151,0,False,False,2022-05-27
4,4,1,307,0,False,False,2022-05-03


In [244]:
campaign.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 7 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   client_id                   41188 non-null  int64 
 1   number_contacts             41188 non-null  int64 
 2   contact_duration            41188 non-null  int64 
 3   previous_campaign_contacts  41188 non-null  int64 
 4   previous_outcome            41188 non-null  bool  
 5   campaign_outcome            41188 non-null  bool  
 6   last_contact_date           41188 non-null  object
dtypes: bool(2), int64(4), object(1)
memory usage: 1.6+ MB


# Guardar datos resultantes en un csv

In [245]:
client.to_csv('client.csv', sep=',', index=False, encoding='utf-8')
campaign.to_csv('campaign.csv', sep=',', index=False, encoding='utf-8')
economics.to_csv('economics.csv', sep=',', index=False, encoding='utf-8')